In [8]:
!pip install -q transformers torch newspaper3k nltk sentencepiece
!pip install -q lxml_html_clean
!pip install -q newspaper3k
!pip install -q trafilatura


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 19.9 MB/s eta 0:00:00


In [16]:
import warnings
warnings.filterwarnings("ignore")


In [9]:
import torch
import nltk
from transformers import pipeline
from newspaper import Article

nltk.download('punkt')

DEVICE = 0 if torch.cuda.is_available() else -1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    tokenizer="sshleifer/distilbart-cnn-12-6",
    device=DEVICE
)


Device set to use cpu


In [11]:
import trafilatura

def extract_text(input_data: str) -> str:
    if input_data.startswith("http"):
        downloaded = trafilatura.fetch_url(input_data)
        return trafilatura.extract(downloaded)
    return input_data


In [12]:
def chunk_text(text, max_words=400):
    words = text.split()
    for i in range(0, len(words), max_words):
        yield " ".join(words[i:i + max_words])


In [13]:
def summarize_article(
    input_data: str,
    max_length=60,
    min_length=35
) -> str:
    text = extract_text(input_data)

    if len(text.strip()) == 0:
        return "❌ No valid text found."

    chunks = list(chunk_text(text))
    summaries = []

    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )[0]["summary_text"]
        summaries.append(summary)

    # Final compression pass → exactly 3 sentences
    final_summary = summarizer(
        " ".join(summaries),
        max_length=75,
        min_length=45,
        do_sample=False
    )[0]["summary_text"]

    return final_summary


In [14]:
url = "https://www.bbc.com/news/articles/c39428dv18yo"
print(summarize_article(url))


 AI model from Google's DeepMind reads recipe for life in DNA . Called AlphaGenome, it could help scientists discover why subtle differences in our DNA put us at risk of conditions such as high blood pressure, dementia and obesity . It could dramatically accelerate our understanding of genetic diseases .


In [17]:
long_text = """
India announced a new policy initiative today aimed at boosting semiconductor manufacturing, marking a significant step in the country’s push toward technological self-reliance and global competitiveness. The move comes as part of a broader strategy to strengthen domestic capabilities in critical technologies and reduce dependence on imported chips, which are essential for sectors ranging from consumer electronics and automobiles to defence and artificial intelligence.

Under the new initiative, the Government of India plans to expand financial incentives, streamline regulatory approvals, and improve infrastructure support for semiconductor fabrication units, packaging facilities, and design ecosystems. The policy also emphasizes collaboration with global technology leaders, encouraging joint ventures and knowledge transfer to accelerate the development of advanced manufacturing processes within the country.

A key focus of the initiative is talent and research. By supporting specialized training programs, strengthening university–industry partnerships, and funding research and development, the government aims to build a sustainable semiconductor ecosystem rather than isolated manufacturing units. The Ministry of Electronics and Information Technology is expected to play a central role in coordinating these efforts across states and private stakeholders.

Industry experts view this policy as a timely intervention, especially amid global supply chain uncertainties. If executed effectively, the initiative could position India as a reliable hub for semiconductor manufacturing, generate high-skill employment, and play a crucial role in the country’s long-term digital and industrial growth.
"""
print(summarize_article(long_text))


Your max_length is set to 75, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


 India announced a new policy initiative aimed at boosting semiconductor manufacturing . The move comes as part of a broader strategy to strengthen domestic capabilities in critical technologies and reduce dependence on imported chips . Industry experts view this policy as a timely intervention, especially amid global supply chain uncertainties .
